In [ ]:
!pip install jschon

In [ ]:
!pip install js2py

In [10]:
from jschon import create_catalog,JSON, JSONSchema
import json,os
import pathlib
import inspect
import  js2py


In [11]:
create_catalog('2019-09')


Catalog('catalog')

In [25]:
keywords_to_avoid = ['not','anyOf','oneOf','allOf','properties','patternPropreties','items']
def extract_errors_map(error:dict, error_map:dict):
    # print(error)
    # if there's errors in subschema treat them since they are the origin of the error  
    keywordLocation = str(error['keywordLocation'])
    key = keywordLocation.split('/')[-1]
    if 'errors' in error:
        if key in  keywords_to_avoid :
            if error_map.get(key) is None : 
                error_map[key]=[error['error'] if 'error' in error else '']
            else : 
                error_map[key].append(error['error'] if 'error' in error else '') 
        for e in error['errors']:
            extract_errors_map(e, error_map)
    else : 
        # if key not in keywords_to_avoid:
        if error_map.get(key) is None : 
            error_map[key]=[error['error'] if 'error' in error else '']
        else : 
            error_map[key].append(error['error'] if 'error' in error else '') 
   

def extract_errors(errors): 
    error_map= {}
    for error in errors : 
        extract_errors_map(error,error_map)
    return list(error_map.keys())

In [13]:
import json
import subprocess
def migrate_schema(schema_migration_path,schema_path):
    try:
        # Convertir le schéma JSON en chaîne JSON
        schema_migration_path_str = json.dumps(schema_migration_path)
        schema_path_str = json.dumps(schema_path)
        # Appeler le script TypeScript avec le schéma JSON en tant qu'argument
        command = ['ts-node', '-e', f'require("./scripts/migrate").migrate({schema_migration_path_str},{schema_path_str})']
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell=True, text=True)
        stdout, stderr = process.communicate()
        # Vérifier si une erreur s'est produite pendant l'exécution du script TypeScript
        if stderr:
            raise RuntimeError(f"Erreur lors de l'exécution du script TypeScript : {stderr}")
        return True

    except Exception as e:
        # print(str(e))
        print(f"Une erreur s'est produite : {e}")
        return None
def migrate(schemas_dir:str,schema_name:str): 
    schema_migration_path = f'{schemas_dir}/{schema_name}-migrated.json'
    schema_path = f'{schemas_dir}/{schema_name}.json'
    try : 
        schema_migrated = migrate_schema(schema_migration_path,schema_path)
        if schema_migrated is None :
            return False 
        return True
    except Exception as e: 
        print(e)
        return False

In [26]:
from tqdm import tqdm
def process_sub_dir(files_list) : 
    for subdir in tqdm(files_list) :
        schema_name = 'schema'
        schema_dir =f'{subdir}'
        print(f"Migrating : {schema_name}") 
        res = migrate(schema_dir,schema_name)
        if res : 
            print("Migrating correctly")
        else : 
            print("Could not migrate")

def migrate_schemas(): 
    files = [f'./dataset/{x}' for x in os.listdir("./dataset")[:-2]]
    process_sub_dir(files)
            

In [27]:
migrate_schemas()

  0%|          | 0/11 [00:00<?, ?it/s]

Migrating : schema


  9%|▉         | 1/11 [00:05<00:52,  5.22s/it]

Migrating correctly
Migrating : schema


 18%|█▊        | 2/11 [00:07<00:32,  3.63s/it]

...

Migrating correctly
Migrating : schema


 91%|█████████ | 10/11 [00:31<00:02,  2.92s/it]

Migrating correctly
Migrating : schema


100%|██████████| 11/11 [00:33<00:00,  3.06s/it]

Migrating correctly


In [16]:

def validate_instance(schema_dir:str,instance_dir): 
    with open(schema_dir,'r',encoding='utf-8') as f : 
        schema= json.load(f)
    print(schema)
    demo_schema = JSONSchema(schema)
    # print(demo_schema)
    with open (instance_dir,'r',encoding='utf-8') as f : 
        instance=json.load(f)
        
    result = demo_schema.evaluate(
        JSON(instance)
    )
    return result 
def get_errors_validation(schema_dir:str,instance_dir:str): 
    result = validate_instance(schema_dir,instance_dir)
    # print(result.output('detailed'))# if errors in result.output('detailed')
    if(result.output("detailed")['valid']==False or "errors" in result.output("detailed")): 
        errors = extract_errors(result.output('detailed')['errors'])
    else : 
        errors=[]
    return errors

In [19]:
src_file_path = os.path.dirname(os.path.abspath("__file__"))
schema_path = src_file_path+'/dataset/10/schema-migrated.json'
instance_path = src_file_path+'/dataset/10/instance.json'
# output = validate_instance(schema_path,instance_path).output('verbose')
# with open(src_file_path+'/dataset/8/errors.json','w') as f: 
#     json.dump(output,f)
result = validate_instance(schema_path,instance_path)
result.output("detailed")
# errors = get_errors_validation(schema_path,instance_path)
# print(errors)

{'additionalProperties': False, 'properties': {'createdAt': {'additionalProperties': False, 'properties': {'since': {'format': 'date-time', 'type': 'string'}, 'until': {'format': 'date-time', 'type': 'string'}}, 'type': 'object'}, 'level': {'additionalProperties': False, 'properties': {'comparison': {'default': 'gte', 'enum': ['eq', 'gt', 'lt', 'gte', 'lte'], 'type': 'string'}, 'value': {'default': 'info', 'enum': ['debug', 'info', 'warn', 'error', 'critical'], 'type': 'string'}}, 'type': 'object'}, 'source': {'additionalProperties': False, 'properties': {'options': {'type': 'string'}, 'pattern': {'type': 'string'}}, 'required': ['pattern'], 'type': 'object'}, 'tags': {'properties': {'match': {'enum': ['all', 'exact', 'any'], 'type': 'string'}, 'value': {'items': {'not': {'const': 'logs'}, 'type': 'string'}, 'type': 'array'}}, 'type': 'object'}, 'time': {'additionalProperties': False, 'properties': {'since': {'format': 'date-time', 'type': 'string'}, 'until': {'format': 'date-time', 't

{'valid': False,
 'instanceLocation': '',
 'keywordLocation': '',
 'absoluteKeywordLocation': 'urn:uuid:9c74c262-a066-4d56-82e1-d161393cf6a9#',
 'errors': [{'instanceLocation': '/tags/value',
   'keywordLocation': '/properties/tags/properties/value/items',
   'absoluteKeywordLocation': 'urn:uuid:9c74c262-a066-4d56-82e1-d161393cf6a9#/properties/tags/properties/value/items',
   'errors': [{'instanceLocation': '/tags/value/0',
     'keywordLocation': '/properties/tags/properties/value/items/type',
     'absoluteKeywordLocation': 'urn:uuid:9c74c262-a066-4d56-82e1-d161393cf6a9#/properties/tags/properties/value/items/type',
     'error': 'The instance must be of type "string"'},
    {'instanceLocation': '/tags/value/1',
     'keywordLocation': '/properties/tags/properties/value/items/type',
     'absoluteKeywordLocation': 'urn:uuid:9c74c262-a066-4d56-82e1-d161393cf6a9#/properties/tags/properties/value/items/type',
     'error': 'The instance must be of type "string"'},
    {'instanceLocation

In [28]:
src_file_path = os.path.dirname(os.path.abspath("__file__"))

# print(src_file_path)
dataset_dir =  src_file_path+'/dataset'
for dir in os.listdir(dataset_dir) : 
    if dir=='instances' or dir=='schemas' : 
        continue 
    print(f"Treating schema : {dir}")
    schema_dir = f'{dataset_dir}/{dir}/schema-migrated.json'
    instance_dir=f'{dataset_dir}/{dir}/instance.json'
    result=validate_instance(schema_dir,instance_dir)
    errors=get_errors_validation(schema_dir,instance_dir)
    
    # print(errors)
    with open(f'{dataset_dir}/{dir}/errors.json','w') as f :
        res = result.output("detailed")
        res['total_errors'] = errors
        json.dump(res,f,indent=2) 

Treating schema : 1
{'required': ['name'], 'properties': {'name': {'anyOf': [{'type': 'object', 'description': 'Cool! We got a string here!', 'required': ['a'], 'properties': {'a': {'type': 'object', 'properties': {'b': {'type': 'string'}}, 'required': ['b']}}}, {'type': 'integer'}]}}, '$schema': 'https://json-schema.org/draft/2019-09/schema'}
{'required': ['name'], 'properties': {'name': {'anyOf': [{'type': 'object', 'description': 'Cool! We got a string here!', 'required': ['a'], 'properties': {'a': {'type': 'object', 'properties': {'b': {'type': 'string'}}, 'required': ['b']}}}, {'type': 'integer'}]}}, '$schema': 'https://json-schema.org/draft/2019-09/schema'}
Treating schema : 10
{'additionalProperties': False, 'properties': {'createdAt': {'additionalProperties': False, 'properties': {'since': {'format': 'date-time', 'type': 'string'}, 'until': {'format': 'date-time', 'type': 'string'}}, 'type': 'object'}, 'level': {'additionalProperties': False, 'properties': {'comparison': {'defa